# 基于TextCNN的头条新闻分类笔记

## 编写配置

In [1]:
import torch 
import torch.nn as nn

config = {
    'train_file_path': '../../../data/toutiao_news_cls/train.csv',
    'test_file_path': '../../../data/toutiao_news_cls/test.csv',
    'train_val_ratio': 0.1,  # 10%用作验证集
    'vocab_size': 10000,   # 词典 3W
    'batch_size': 32,      # batch 大小 64
    'num_epochs': 10,      # 10次迭代
    'learning_rate': 1e-3, # 学习率
    'logging_step': 300,   # 每跑300个batch记录一次
    'seed': 2022           # 随机种子
}

config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu' # cpu&gpu

import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2022

## 获得词汇表

In [2]:
from collections import Counter
from tqdm import tqdm
import jieba

def get_vocab(config):
    token_counter = Counter()
    with open(config['train_file_path'], 'r', encoding='utf8') as f:
        lines = f.readlines()
        for line in tqdm(lines, desc='Counting tokens...', total=len(lines)):
            sent = line.split(',')[-1].strip()
            sent_cut = list(jieba.cut(sent))
            token_counter.update(sent_cut)
            
    vocab = set(token for token, _ in token_counter.most_common(config['vocab_size']))
    return vocab

In [3]:
vocab = get_vocab(config)
vocab

Counting tokens...:   0%|                                         | 0/53361 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/19/dgxwsbgd76728hx577833nx00000gn/T/jieba.cache
Loading model cost 0.729 seconds.
Prefix dict has been built successfully.
Counting tokens...: 100%|███████████████████████████| 53361/53361 [00:06<00:00, 7934.29it/s]


{'金庸',
 '险',
 '个股',
 '们',
 '支队',
 '这座',
 '可达',
 '余年',
 '驾考',
 '旅游',
 '大好',
 'Apple',
 '全方位',
 '向往',
 '功勋',
 '范冰冰',
 '自拍',
 '论坛',
 '莫斯科',
 '上单',
 '郜林',
 '4.0',
 '涨停',
 '追',
 '忽而',
 '地块',
 '曝出',
 '在京举行',
 '主持',
 '曾经',
 '反对派',
 '厂长',
 '犹太人',
 '排名',
 '猪价',
 '许多',
 '貂蝉',
 '造假',
 '全城',
 '加强',
 '活动',
 '禁令',
 '读懂',
 '却说',
 '桑葚',
 '户外',
 '二维码',
 '印',
 '挽回',
 '扬州',
 'OMG',
 '声明',
 '280',
 '非法',
 '失踪',
 '神剧',
 '灾区',
 '人民日报',
 '维权',
 '旅行',
 '必须',
 '上山',
 '真题',
 '居然',
 '拍卖',
 '就够',
 '岳母',
 '手榴弹',
 '吸引',
 '毛',
 '苹果',
 '升',
 '不动产',
 '河南',
 '算是',
 '强大',
 '四川省',
 '科目',
 '孔',
 '居民',
 '续航',
 ' ',
 '现在',
 '六亲',
 '网购',
 '内蒙古',
 '地标',
 '史',
 '地点',
 '吸金',
 '舆论',
 '挑战者',
 '变动',
 '卡车',
 '碎',
 '校长',
 '全能',
 '想到',
 '雪域',
 '开战',
 '可怜',
 '卡',
 '故乡',
 '700',
 '主裁',
 '德云色',
 '廉价',
 '临时',
 '国安',
 '国乒',
 '列车',
 '挖出',
 '骂',
 '封号',
 '早上',
 '超过',
 '陈奕迅',
 '供',
 '方面',
 '外媒',
 '如何',
 '居',
 '行人',
 '投篮',
 '天文',
 '运动会',
 '禁赛',
 '老板',
 '下令',
 '拿下',
 '获得',
 '搭载',
 '形态',
 '干预',
 '抱怨',
 '帮助',
 '小时候',
 '我们',
 '泪目',
 '宣布',
 '长码',
 

## 获得词向量

In [4]:
import bz2

with bz2.open('../../../pt/sgns_weibo/sgns.weibo.word.bz2') as f:
    token_vector = f.readlines()    

In [5]:
for i, line in enumerate(token_vector):
    line_split = line.split()
    if i == 0:
        print(f'词汇规模:{(int)(line_split[0])}, 特征维度:{(int)(line_split[1])}')
    else:
        print(f"词汇:{line_split[0].decode('utf8')}\n特征:{line_split[1:]}")
    print()
    
    if i > 2:
        break

词汇规模:195202, 特征维度:300

词汇:，
特征:[b'0.094386', b'-0.200944', b'-0.030828', b'0.277130', b'-0.074674', b'0.239691', b'0.345185', b'0.298053', b'-0.026679', b'-0.077352', b'0.245854', b'-0.075995', b'0.058849', b'-0.176755', b'-0.072721', b'-0.246469', b'-0.182233', b'0.267472', b'0.109897', b'-0.266215', b'0.176660', b'-0.101581', b'0.374443', b'0.304319', b'0.024169', b'-0.158494', b'-0.254765', b'-0.265324', b'0.225498', b'-0.126215', b'-0.207172', b'-0.334429', b'-0.003584', b'-0.065530', b'0.174206', b'-0.097252', b'0.114595', b'-0.158193', b'-0.099948', b'-0.145332', b'-0.076565', b'-0.119460', b'0.074718', b'-0.056694', b'0.394867', b'-0.127122', b'0.270542', b'0.087482', b'-0.114715', b'0.124381', b'-0.149974', b'0.192933', b'0.131023', b'0.099271', b'-0.062209', b'0.079382', b'0.000578', b'-0.166215', b'0.155815', b'-0.279675', b'-0.184237', b'0.123209', b'0.043216', b'-0.308681', b'0.036135', b'0.064121', b'0.037674', b'0.026703', b'0.307341', b'0.089780', b'-0.057681', b'-0.2523

In [6]:
def get_embedding(vocab):
    token2embedding ={}

    with bz2.open('../../../pt/sgns_weibo/sgns.weibo.word.bz2') as f:
        token_vector = f.readlines()

        meta_info = token_vector[0].split()
        print(f'{(int)(meta_info[0])} tokens in embedding file in total, vector size is {(int)(meta_info[1])}')

        for line in tqdm(token_vector[1:]):
            line = line.split()
            token = line[0].decode('utf8')

            vector = line[1:]

            if token in vocab:
                token2embedding[token] = [float(num) for num in vector]

        # enumerate(, [start])
        token2id = {token: idx for idx, token in enumerate(token2embedding.keys(), 4)}
        id2embedding = {token2id[token]: embedding for token, embedding in token2embedding.items()}

        PAD, UNK, BOS, EOS = '<pad>', '<unk>', '<bos>', '<eos>'

        token2id[PAD] = 0
        token2id[UNK] = 1
        token2id[BOS] = 2
        token2id[EOS] = 3

        id2embedding[0] = [.0] * int(meta_info[1])
        id2embedding[1] = [.0] * int(meta_info[1])

        id2embedding[2] = np.random.random(int(meta_info[1])).tolist()
        id2embedding[3] = np.random.random(int(meta_info[1])).tolist()

        emb_mat = [id2embedding[idx] for idx in range(len(id2embedding))]

        return torch.tensor(emb_mat, dtype=torch.float), token2id, len(vocab)+4


In [7]:
emb_mat, token2id, config['vocab_size'] = get_embedding(vocab)

195202 tokens in embedding file in total, vector size is 300


100%|████████████████████████████████████████████| 195202/195202 [00:03<00:00, 60309.12it/s]


## 数据预处理并编写DataLoader

In [8]:
def tokenizer(sent, token2id):
    ids = [token2id.get(token, 1) for token in jieba.cut(sent)]
    return ids

In [9]:
import pandas as pd
from collections import defaultdict

In [10]:
def read_data(config, token2id, mode='train'):
    
    data_df = pd.read_csv(config[f'{mode}_file_path'], sep=',')
    LABEL, SENTENCE = 'label', 'sentence'
    data_df[SENTENCE].apply(lambda s: ' '.join([str(id_) for id_ in tokenizer(s, token2id)]))
    data_df['input_ids'] = data_df[SENTENCE].apply(lambda s: ' '.join([str(id_) for id_ in tokenizer(s, token2id)]))
    sentence_wids = np.array([[int(id_) for id_ in v.split()] for v in data_df['input_ids'].values])

    if mode == 'train':
        labels = data_df[LABEL].values
        
        X_train, y_train = defaultdict(list), []
        X_val, y_val = defaultdict(list), []
        num_val = int(config['train_val_ratio'] * len(data_df))
        
        # shuffle ids
        ids = np.random.choice(range(len(data_df)), size=len(data_df), replace=False)
        train_ids = ids[num_val:]
        val_ids = ids[:num_val]
        
        # get input_ids
        X_train['input_ids'], y_train = sentence_wids[train_ids], labels[train_ids]
        X_val['input_ids'], y_val = sentence_wids[val_ids], labels[val_ids]
        
        # label 
        label2id = {label: i for i, label in enumerate(np.unique(y_train))}
        id2label = {i: label for label, i in label2id.items()}
        y_train = torch.tensor([label2id[y] for y in y_train], dtype=torch.long)
        y_val = torch.tensor([label2id[y] for y in y_val], dtype=torch.long)

        return X_train, y_train, X_val, y_val, label2id, id2label

    else:
        X_test = defaultdict(list)
        X_test['input_ids'] = sentence_wids
        y_test = torch.zeros(len(data_df), dtype=torch.long)
        
        return X_test, y_test

In [11]:
# X_train, y_train, X_val, y_val, label2id, id2label = read_data(config, token2id, mode='train')

In [12]:
# X_test, y_test = read_data(config, token2id, mode='test')

#### Dataset提供数据集的封装，创建/继承Dataset必须实现:
+ __len__: 整个数据集的长度
+ __getitem__: 支持数据集索引的函数

In [13]:
from torch.utils.data import Dataset

class TNEWSDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y
        
    def __getitem__(self, idx):
        return {
            'input_ids': self.x['input_ids'][idx],
            'label': self.y[idx]
        }
    
    def __len__(self):
        return self.y.size(0)

#### 使用DataLoader实现数据集的并行加载
+ DataLoader提供一个可迭代对象，实现数据并行加载，从TNEWSDataset返回一个example，取多次，最后形成一个长度为batch_size的列表examples
+ examples的格式：[dict1, dict2, ...]
+ collate_fn()将examples中的数据合并为Tensor

In [14]:
def collate_fn(examples):
    input_ids_lst = []
    labels = []
    for example in examples:
        input_ids_lst.append(example['input_ids'])
        labels.append(example['label'])
        
    # input_ids_lst中最长的句子长度
    max_length = max(len(input_ids) for input_ids in input_ids_lst)
    # 定义一个Tensor
    input_ids_tensor = torch.zeros((len(labels), max_length), dtype=torch.long)
    
    for i, input_ids in enumerate(input_ids_lst):
        seq_len = len(input_ids)
        input_ids_tensor[i, :seq_len] = torch.tensor(input_ids, dtype=torch.long)
    
    return {
        'input_ids': input_ids_tensor,
        'label': torch.tensor(labels, dtype=torch.long)
    }

In [15]:
from torch.utils.data import DataLoader

def build_dataloader(config, vocab):
    X_train, y_train, X_val, y_val, label2id, id2label = read_data(config, token2id, mode='train')
    X_test, y_test = read_data(config, token2id, mode='test')
    
    train_dataset = TNEWSDataset(X_train, y_train)
    val_dataset = TNEWSDataset(X_val, y_val)
    test_dataset = TNEWSDataset(X_test, y_test)
    
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=config['batch_size'], num_workers=0, shuffle=True, collate_fn=collate_fn)
    val_dataloader = DataLoader(dataset=val_dataset, batch_size=config['batch_size'], num_workers=0, shuffle=False, collate_fn=collate_fn)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=config['batch_size'], num_workers=0, shuffle=False, collate_fn=collate_fn)

    return id2label, train_dataloader, val_dataloader, test_dataloader

In [16]:
id2label, train_dataloader, val_dataloader, test_dataloader = build_dataloader(config, vocab)

/Users/fenghaoguo/opt/anaconda3/envs/ng/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys


In [17]:
for batch in train_dataloader:
    print(len(batch['input_ids']))
    print(batch)
    break

32
{'input_ids': tensor([[ 368,  749,  323,  624,   48,    1, 5674, 1957,  892, 3675,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [8462,    1,  739,   34,   18,    1,   22,   56, 1411,   38,    8,    7,
           33,    1, 1411,  670,  260,   15,    1,    1,    7],
        [9009,    1,    1, 2188,    4,    1,    1,    1, 2998,    1,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [5197,    1,    1, 5840,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [ 340, 4981, 7492,  846,   17,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   1,  492, 4721,   46,    1,    4,    1,  314,    1,  153,   12, 3680,
            5,  143,    0,    0,    0,    0,    0,    0,    0],
        [  15,  879,   41,    1,   32,  563,    4,    1,    1,    1, 1034,    8,
          114,   17,    0,    0,

## 实现TextCNN并训练验证

In [18]:
model_config = {
    'embedding_pretrained' : emb_mat,
    'num_filters' : 256,
    'emb_size' : emb_mat.shape[1],
    'dropout' : 0.3,
    'filter_sizes' : [2,3,5],
    'num_classes' : len(id2label)
}

In [19]:
emb_mat.dtype

torch.float32

In [20]:
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self, config):
        super(Model, self).__init__()
        
        self.embedding = nn.Embedding.from_pretrained(config['embedding_pretrained'], freeze=True)
        self.convs = nn.ModuleList([nn.Conv2d(1, config['num_filters'], (k, config['emb_size'])) for k in config['filter_sizes']])
        self.dropout = nn.Dropout(config['dropout'])
        
        self.fc = nn.Linear(len(config['filter_sizes']) * config['num_filters'], config['num_classes'])

        
    def convs_and_pool(self, x, conv):
        # x [batch_size, out_channels, seq_len_out, 1]
        # x [batch_size, out_channels, seq_len_out]
        x = F.relu(conv(x)).squeeze(3)
        
        # x [batch_size, out_channels, 1]
        # x [batch_size, out_channels]
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        
        return x
    
    def forward(self, input_ids=None, label=None):
        # out [batch_size, seq_len, embedding_dim]
        out = self.embedding(input_ids)
        
        # out [batch_size, 1, seq_len, embedding_dim]
        out = out.unsqueeze(1)
        
        # (batch_size, out_channels)
        out = torch.cat([self.convs_and_pool(out, conv) for conv in self.convs], 1)
        
        out = self.dropout(out)
        out = self.fc(out)
        output = (out, )
        
        if label is not None:  # for train
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(out, label)
            output = (loss,) + output
            
        return output

In [21]:
model = Model(model_config)

In [22]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def evaluation(config, model, val_dataloader):
    model.eval()
    preds = []
    labels = []
    val_loss = 0.
    val_iterator = tqdm(val_dataloader, desc='Evaluation...', total=len(val_dataloader))
    with torch.no_grad():
        for batch in val_iterator:
            labels.append(batch['label'])
            batch = {item:value.to(config['device']) for item, value in batch.items()}
            
            # val output (loss, out)
            loss, logits = model(**batch)[:2]
            val_loss += loss.item()
            
            preds.append(logits.argmax(dim=-1).detach().cpu())
            
    avg_val_loss = val_loss/len(val_dataloader)
    labels = torch.cat(labels, dim=0).numpy()
    preds = torch.cat(preds, dim=0).numpy()
    
    f1 = f1_score(labels, preds, average='macro')
    precision = precision_score(labels, preds, average='macro')
    recall = recall_score(labels, preds, average='macro')
    accuracy = accuracy_score(labels, preds)

    return avg_val_loss, f1, precision, recall, accuracy

In [27]:
import pickle
from torch.optim import AdamW
from tqdm import trange

def train(model, config, id2label, train_dataloader, val_dataloader):
    optimizer = AdamW(model.parameters(), lr=config['learning_rate'])
    model.to(config['device'])
    epoches_iterator = trange(config['num_epochs'])
    
    # 200batch/epoch * 10epochs = global_step
    global_step = 0
    train_loss = 0.
    logging_loss = 0.
    
    best_f1 = 0.
    best_precision = 0.
    best_recall = 0.
    best_accuracy = 0.
    
    for epoch in epoches_iterator:
        train_iterator = tqdm(train_dataloader, desc='Training', total=len(train_dataloader))
        model.train()
        for batch in train_iterator:
            batch = {item:value.to(config['device']) for item, value in batch.items()}
            
            # train output (loss, out)
            loss = model(**batch)[0]
            
            model.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss
            global_step += 1
            
            if global_step % config['logging_step'] == 0:
                print_train_loss = train_loss / config['logging_step']
                train_loss = 0.
                avg_val_loss, f1, precision, recall, accuracy = evaluation(config, model, val_dataloader)

                if best_f1 < f1:
                    best_f1 = f1
                    best_precision = precision
                    best_recall = recall
                    best_accuracy = accuracy
                    print_log = f'''>>> training loss: {print_train_loss: .4f}, valid loss: {avg_val_loss: .4f}\n
                            valid f1 score: {f1: .4f}, valid precision score: {precision: .4f},
                            valid recall score: {recall: .4f}, valid accuracy score: {accuracy: .4f}'''
                    print(print_log)
                    with open('../../../pt_tmp/cls/textcnn_base_chinese/pytorch_model.pkl', 'wb') as f:
                        pickle.dump(model, f)

                model.train()
                
    return best_f1, best_precision, best_recall, best_accuracy

In [28]:
f1, precision, recall, accuracy = train(model, config, id2label, train_dataloader, val_dataloader)
print_log = f'''valid f1 score: {f1: .4f}, valid precision score: {precision: .4f},
                valid recall score: {recall: .4f}, valid accuracy score: {accuracy: .4f}'''
print(print_log)

Training:  11%|████▋                                     | 166/1501 [00:10<01:20, 16.61it/s]


Training:  20%|████████▎                                 | 298/1501 [00:19<01:14, 16.10it/s]

Evaluation...:   0%|                                                | 0/167 [00:00<?, ?it/s]

Evaluation...:   2%|▉                                       | 4/167 [00:00<00:04, 35.65it/s]

Evaluation...:   5%|█▉                                      | 8/167 [00:00<00:04, 36.30it/s]

Evaluation...:   7%|██▊                                    | 12/167 [00:00<00:04, 37.19it/s]

Evaluation...:  10%|███▉                                   | 17/167 [00:00<00:03, 38.62it/s]

Evaluation...:  13%|████▉                                  | 21/167 [00:00<00:03, 38.82it/s]

Evaluation...:  15%|█████▊                                 | 25/167 [00:00<00:03, 38.29it/s]

Evaluation...:  17%|██████▊                                | 29/167 [00:00<00:03, 37.05it/s]

Evaluation...:  20%|███████▋                               | 33/167 [00:00<00:03, 37.20it/s]

Evaluation...:  22%|████████▋                              |

Evaluation...:  45%|█████████████████▌                     | 75/167 [00:02<00:02, 37.88it/s]

Evaluation...:  47%|██████████████████▍                    | 79/167 [00:02<00:02, 38.28it/s]

Evaluation...:  50%|███████████████████▍                   | 83/167 [00:02<00:02, 38.60it/s]

Evaluation...:  52%|████████████████████▎                  | 87/167 [00:02<00:02, 36.75it/s]

Evaluation...:  54%|█████████████████████▎                 | 91/167 [00:02<00:02, 36.22it/s]

Evaluation...:  57%|██████████████████████▏                | 95/167 [00:02<00:02, 35.92it/s]

Evaluation...:  59%|███████████████████████                | 99/167 [00:02<00:01, 36.00it/s]

Evaluation...:  62%|███████████████████████▍              | 103/167 [00:02<00:01, 36.76it/s]

Evaluation...:  64%|████████████████████████▎             | 107/167 [00:02<00:01, 37.40it/s]

Evaluation...:  67%|█████████████████████████▍            | 112/167 [00:03<00:01, 38.90it/s]

Evaluation...:  69%|██████████████████████████▍           | 

>>> training loss:  1.4154, valid loss:  1.4659

                            valid f1 score:  0.4721, valid precision score:  0.5316,
                            valid recall score:  0.4596, valid accuracy score:  0.5208



Training:  32%|█████████████▏                            | 473/1501 [00:34<01:08, 14.97it/s]


...

Training:  79%|████████████████████████████████▎        | 1185/1501 [01:28<00:18, 16.94it/s]


Training:  79%|████████████████████████████████▍        | 1189/1501 [01:29<00:17, 17.57it/s]

Evaluation...:   0%|                                                | 0/167 [00:00<?, ?it/s]

Evaluation...:   2%|▉                                       | 4/167 [00:00<00:04, 36.14it/s]

Evaluation...:   5%|█▉                                      | 8/167 [00:00<00:04, 35.53it/s]

Evaluation...:   7%|██▊                                    | 12/167 [00:00<00:04, 35.93it/s]

Evaluation...:  10%|███▋                                   | 16/167 [00:00<00:04, 37.07it/s]

Evaluation...:  12%|████▋                                  | 20/167 [00:00<00:03, 37.07it/s]

Evaluation...:  14%|█████▌                                 | 24/167 [00:00<00:03, 37.67it/s]

Evaluation...:  17%|██████▌                                | 28/167 [00:00<00:03, 36.62it/s]

Evaluation...:  19%|███████▍                               | 32/167 [00:00<00:03, 36.41it/s]

Evaluation...:  22%|████████▍                              |

Training:  96%|███████████████████████████████████████▎ | 1437/1501 [01:48<00:03, 17.54it/s]


Training:  99%|████████████████████████████████████████▋| 1489/1501 [01:51<00:00, 16.28it/s]

Evaluation...:   0%|                                                | 0/167 [00:00<?, ?it/s]

Evaluation...:   2%|▉                                       | 4/167 [00:00<00:04, 35.58it/s]

Evaluation...:   5%|█▉                                      | 8/167 [00:00<00:04, 36.32it/s]

Evaluation...:   8%|███                                    | 13/167 [00:00<00:03, 39.00it/s]

Evaluation...:  11%|████▏                                  | 18/167 [00:00<00:03, 40.48it/s]

Evaluation...:  14%|█████▎                                 | 23/167 [00:00<00:03, 42.17it/s]

Evaluation...:  17%|██████▌                                | 28/167 [00:00<00:03, 41.23it/s]

Evaluation...:  20%|███████▋                               | 33/167 [00:00<00:03, 41.78it/s]

Evaluation...:  23%|████████▊                              | 38/167 [00:00<00:03, 41.18it/s]

Evaluation...:  26%|██████████                             |

valid f1 score:  0.5188, valid precision score:  0.5395,
                valid recall score:  0.5077, valid accuracy score:  0.5266


## 预测并保存结果

In [31]:
def predict(config, id2label, model, test_dataloader):
    test_iterator = tqdm(test_dataloader, desc='Testing', total=len(test_dataloader))
    model.eval()
    test_preds = []
    
    with torch.no_grad():
        for batch in test_iterator:
            batch = {item: value.to(config['device']) for item, value in batch.items()}

            logits = model(**batch)[1]
            test_preds.append(logits.argmax(dim=-1).detach().cpu())
            
    test_preds = torch.cat(test_preds, dim=0).numpy()
    test_preds = [id2label[id_] for id_ in test_preds]
        
    test_df = pd.read_csv(config['test_file_path'], sep=',')
    # test_df.insert(1, column=['label_pred'], value=test_preds)
    test_df['label_pred'] = test_preds
    # test_df.drop(columns=['sentence'], inplace=True)
    test_df.to_csv('submission.csv', index=False, encoding='utf8')

In [32]:
predict(config, id2label, best_model, test_dataloader)

Testing: 100%|████████████████████████████████████████████| 313/313 [00:07<00:00, 40.90it/s]


In [26]:
test_df = pd.read_csv(config['test_file_path'], sep=',')

In [33]:
train_df = pd.read_csv(config['train_file_path'], sep=',')

In [50]:
train_df.head(10)

,id,label,label_desc,sentence
0,0,108,news_edu,上课时学生手机响个不停，老师一怒之下把手机摔了，家长拿发票让老师赔，大家怎么看待这种事？
1,1,104,news_finance,商赢环球股份有限公司关于延期回复上海证券交易所对公司2017年年度报告的事后审核问询函的公告
2,2,106,news_house,通过中介公司买了二手房，首付都付了，现在卖家不想卖了。怎么处理？
3,3,112,news_travel,2018年去俄罗斯看世界杯得花多少钱？
4,4,109,news_tech,剃须刀的个性革新，雷明登天猫定制版新品首发
5,5,103,news_sports,再次证明了“无敌是多么寂寞”——逆天的中国乒乓球队！
6,6,109,news_tech,三农盾SACC-全球首个推出：互联网+区块链+农产品的电商平台
7,7,116,news_game,重做or新英雄？其实重做对暴雪来说同样重要
8,8,103,news_sports,如何在商业活动中不受人欺骗？
9,9,101,news_culture,87版红楼梦最温柔的四个丫鬟，娶谁都是一生的福气


In [39]:
train_df['label'].unique()

array([108, 104, 106, 112, 109, 103, 116, 101, 107, 100, 102, 110, 115,
       113, 114])